# import bags of words 

In [80]:
import requests, re
from bs4 import BeautifulSoup


with open('stopwords/polish.stopwords.txt') as f:
    polish_sw = [word.replace('\n','') for word in f.readlines()]
    

class Feed:
    def __init__(self, name, url):
        self.name = name
        self.url = url
    
    def makeSoup(self):
        page = requests.get(self.url)
        soup = BeautifulSoup(page.content, 'html.parser')
        return soup
    
    
    def getHeadersSport(self):
        
        soup = self.makeSoup()
        
        if self.name == 'Onet':
            divs = soup.find_all(class_="sectionLine sectionLineMax")
            articles_sets = [div.find_all('a') for div in divs]
            onet_dic = {}
            for articles in articles_sets:
                for article in articles:                    
                    onet_dic[article.find(class_="title").text] = article.attrs.get('href')
            return onet_dic  
        
        if self.name == 'Onet Sport':
            news = soup.find_all(class_='boxTitle')
            onetsport_dic = {'name' : self.name}
            for i in news:
                onetsport_dic[i.text.strip()] = i.attrs.get('href')
            return onetsport_dic
    
    

    def getHeaders(self):
        
        soup = self.makeSoup()
        
        if self.name == 'Onet':
            divs = soup.find_all(class_="boxContent")
            articles_sets = [div.find_all('a') for div in divs]
            onet_dic = {'name' : self.name}     
            for articles in articles_sets:
                for article in articles:                    
                    onet_dic[article.find(class_="title").text] = article.attrs.get('href')
            return onet_dic   
        
        
        if self.name == 'Wirtualna Polska':
            divs = soup.find_all(class_="sc-1fu2hk8-0 jIlknD")
            articles_sets = [div.find_all('a') for div in divs]
            wp_dic = {'name' : self.name}         
            for articles in articles_sets:
                for article in articles:                
                    if len(wp_dic) <= news_num+1:
                        try:
                            wp_dic[article.find_all('div')[1].text] = article.attrs.get('href')
                        except IndexError:
                            wp_dic[article.find('div').text] = article.attrs.get('href')
                    else:
                        return wp_dic
                    
        
        if self.name == 'W Polityce':
            divs = soup.find_all(class_="nu-lead-articles") + soup.find_all(class_="nu-tile-container nu-main-col--5")
            articles_sets = [div.find_all('a') for div in divs]
            wpolityce_dic = {'name' : self.name}
            for articles in articles_sets:
                for article in articles:
                    if len(wpolityce_dic) <= news_num+1:
                        wpolityce_dic[article.find(class_="short-title").text] = str(self.url[:-1])+str(article.attrs.get('href'))
                    else:
                        return wpolityce_dic
                    


        
onet = Feed('Onet', 'https://www.onet.pl/')
onetsport = Feed('Onet Sport', 'https://sport.onet.pl/')
wp = Feed('Wirtualna Polska', 'https://www.wp.pl/')
wpolityce = Feed('W Polityce', 'https://wpolityce.pl/')
sportpl = Feed('Sport', 'http://www.sport.pl/sport/0,168452.html?str=0_24519889')


feed_list = [onet, wp, wpolityce, sportpl]





def writeOnetSport():
    
    with open('onet_sport.txt', 'a+') as f:
        existing_titles = f.readlines()
        for title, link in onetsport.getHeadersSport().items():
            if title not in existing_titles:
                f.write(title+'\n')

def writePolitics():
    a = 0
    with open('onet_polityka.txt', 'a+') as f:
        existing_titles = f.readlines()
        for title, link in onet.getHeaders().items():
            if a<=12:
                title = title.strip()
                if title not in existing_titles:
                    if title != 'name':
                        f.write(title+'\n')

writeOnetSport()                      


extract key words from article headlines

In [81]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
import numpy as np
import pandas as pd
import re

with open('stopwords/polish.stopwords.txt') as f:
    polish_sw = [word.replace('\n','') for word in f.readlines()]
    
with open('onet_sport.txt') as f:
    docs = f.read().split('\n')[:-1]
    

def pre_process(text):
    
    text=text.lower()
    text=re.sub("<!--?.*?-->","",text)
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text


def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results



docs = list(map(lambda x:pre_process(x), docs))


cv=CountVectorizer(max_df=0.85,stop_words=polish_sw)
word_count_vector=cv.fit_transform(docs)
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

# you only needs to do this once, this is a mapping of index to 
feature_names=cv.get_feature_names()
 
# get the document that we want to extract keywords from
# doc=corpus[1]
 
#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform(corpus))
 
#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())
 
#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,30)

for k,v in keywords.items():
    print(k,v)


starcie 0.681
zapisał 0.577
lewandowski 0.577
historii 0.577
wiadomości 0.562
dobre 0.562
turcji 0.519
grozy 0.519
chwile 0.519
rewolucji 0.511
ratunek 0.511
niechciani 0.511
świata 0.5
zwycięstwo 0.5
wicemistrz 0.5
stocha 0.5
skomentował 0.5
małysz 0.5
kubicy 0.5
kadrze 0.5
jerzego 0.5
debiuty 0.5
brzęczka 0.5
bolesne 0.5
wielki 0.5
transferowy 0.5
problemem 0.5
oficjalnie 0.5
ofensywa 0.5
niemieckie 0.5
